In [1]:
import numpy as np
import optuna
import os
import pandas as pd
import time

working_dir = os.getcwd()
print(working_dir)


/home/rachtrx/mambaforge/envs/sc1015/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/mnt/c/Users/rachm/OneDrive/projects/personal_projects/sc2001/project_1


In [2]:
ARR_LENS = list(range(0, 1_000_001, 100_000))
S_VALUES = list(range(10, 101, 10)) # 10 to 100

In [3]:
class DataLoader:
    def __init__(self, arr_len):
        self.arr_len = arr_len
        
    def generate_data(self):
        return np.random.randint(1, self.arr_len, self.arr_len)

In [4]:
class Model:

    def __init__(self, array, S):
        self.S = S   
        self.array = array

    def train(self):
        self.execution_time = None
        self.key_cmp = 0
        start_time = time.perf_counter()
        self.merge_insertion_sort(0, len(self.array)-1)
        end_time = time.perf_counter()
        self.execution_time = end_time - start_time

    def merge(self, n, m):
        mid = (n + m) // 2
        left = self.array[n:mid + 1]
        right = self.array[mid + 1:m + 1]

        i = j = 0
        k = n

        while i < len(left) and j < len(right):
            self.key_cmp += 1
            if left[i] <= right[j]:  # Stable sort: left[i] goes first if equal
                self.array[k] = left[i]
                i += 1
            else:
                self.array[k] = right[j]
                j += 1
            k += 1

        while i < len(left):
            self.array[k] = left[i]
            i += 1
            k += 1

        while j < len(right):
            self.array[k] = right[j]
            j += 1
            k += 1

    def merge_insertion_sort(self, n, m):
        
        if (m-n <= 0):
            return
        
        if m-n+1 <= self.S:
            self.insertion_sort(n, m)
            return
        
        mid = (n+m)//2
        if (m-n>1):
            self.merge_insertion_sort(n, mid)
            self.merge_insertion_sort(mid+1, m)
        
        self.merge(n, m)

    def merge_sort(self, n, m):
        if (m-n <= 0):
            return
        
        mid = (n+m)//2
        if (m-n>1):
            self.merge_sort(n, mid)
            self.merge_sort(mid+1, m)
        
        self.merge(n, m)

    def insertion_sort(self, n, m):
        i = n+1
        while i <= m:
            j = i
            while j > n:
                self.key_cmp += 1
                if self.array[j] < self.array[j-1]:
                    tmp = self.array[j]
                    self.array[j] = self.array[j-1]
                    self.array[j-1] = tmp
                    j -= 1
                else:
                    break
            i+=1

In [5]:
dataloader = DataLoader(arr_len=100000)
array = dataloader.generate_data()
model = Model(array, 30)
model.train()

result = {
    "Array Length": 100000,
    "Threshold": 30,
    "Execution Time": model.execution_time,
    "Key Comparisons": model.key_cmp
}

print(result)

{'Array Length': 100000, 'Threshold': 30, 'Execution Time': 375.7347978730104, 'Key Comparisons': 1822856}


In [6]:
# results = []

# for arr_len in ARR_LENS:
#     print(f"Running for arrays of length {arr_len}")
#     dataloader = DataLoader(arr_len=arr_len)
#     array = dataloader.generate_data()
    
#     for S in S_VALUES:
        
#         model = Model(array, S)
#         model.train()
        
#         results.append({
#             "Array Length": arr_len,
#             "Threshold": S,
#             "Execution Time": model.execution_time,
#             "Key Comparisons": model.key_cmp
#         })

# df_results = pd.DataFrame(results)

# df_results.to_parquet("sorting_experiment_results.parquet", index=False)

In [7]:
# PARALLEL

# from multiprocessing import Pool, cpu_count
# from tqdm import tqdm
# import pandas as pd

# def run_experiment(args):
#     arr_len, S = args
#     dataloader = DataLoader(arr_len=arr_len)
#     array = dataloader.generate_data()

#     model = Model(array, S)
#     model.train()

#     return {
#         "Array Length": arr_len,
#         "Threshold": S,
#         "Execution Time": model.execution_time,
#         "Key Comparisons": model.key_cmp
#     }

# experiments = [(arr_len, S) for arr_len in ARR_LENS for S in S_VALUES]

# with Pool(cpu_count()) as pool:
#     results = list(tqdm(pool.imap(run_experiment, experiments), total=len(experiments), desc="Processing Arrays"))

# df_results = pd.DataFrame(results)
# df_results.to_parquet("results.parquet", index=False)

# print("Execution complete")
